<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#What-is-a-Jupyter-notebook-?" data-toc-modified-id="What-is-a-Jupyter-notebook-?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>What is a Jupyter notebook ?</a></span></li><li><span><a href="#What-is-Julia-?" data-toc-modified-id="What-is-Julia-?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>What is Julia ?</a></span></li><li><span><a href="#What-is-JuMP-?" data-toc-modified-id="What-is-JuMP-?-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>What is JuMP ?</a></span></li><li><span><a href="#Constructing-linear-programm" data-toc-modified-id="Constructing-linear-programm-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Constructing linear programm</a></span><ul class="toc-item"><li><span><a href="#A-first-LP" data-toc-modified-id="A-first-LP-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>A first LP</a></span></li><li><span><a href="#A-more-complex-LP" data-toc-modified-id="A-more-complex-LP-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>A more complex LP</a></span></li><li><span><a href="#Declare-multiple-variables-/-constraints" data-toc-modified-id="Declare-multiple-variables-/-constraints-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Declare multiple variables / constraints</a></span></li></ul></li></ul></div>

# A quick review of Jupyter / Julia / JuMP

## What is a Jupyter notebook ?

A Jupyter notebook is a document containing 
+ text
  - that we can formatted with Markdown
  - that can contains $\LaTeX$
+ code
  - that we can execute from the browser
  
A notebook is made of cells, each being either text or code
A few tricks :
+ double-click to go in a cell
+ M / Y to change cell type
+ Ctrl-enter to execute a cell
+ shift-enter to execute a cell and go to the next

You can download the .ipynb file through "file" tab at the top left. You can also download a pdf.

## What is Julia ?

A Python like programming language. It is recent and develloped for scientific calculus. 

A few interesting points :
+ open-source
+ Just-in-time compilation
+ has a shell like python
+ ...

Let's make our first steps in Julia. Execute the following cells (shift-enter), don't hesitate to modify a bit:

In [ ]:
1+1

In [ ]:
a = [0 5 10 15]
a[1]

In [ ]:
sum(a)

In [ ]:
sum(x^2 for x in a)

In [ ]:
length(a)

In [ ]:
exp.(a) .- a #to apply an operation or function componentwise just add .  

In [ ]:
@doc exp # to get documentation on a function

In [ ]:
for i = 1:5
    println("itération ",i)
end

In [ ]:
function factoriel(n)
    if n == 0
        return 1
    end
    res = 1
    for i=1:n
        res = res * i
    end
    return res
end

In [ ]:
factoriel(5)

In [ ]:
round(1.9453;sigdigits=2)

In [ ]:
rand(3)

Further info can be found [here](https://learnxinyminutes.com/docs/julia/) or in the [documentation](https://docs.julialang.org/en/v1/)



## What is JuMP ?

JuMP is one of Julia's prominent package.

It is a modeler that allow to easily write optimisation problem and call solver on it.

More information on http://www.juliaopt.org/JuMP.jl/v0.20.0/quickstart/

Let start by installing some packages.

In [ ]:
using Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")

We can list the installed package with the following command.

In [ ]:
Pkg.status()

We now load the packages.

In [ ]:
using JuMP, GLPK


## Constructing linear programm

### A first LP

We want to solve the following LP
$$ \begin{align*} 
\min_{x,y} \quad & 2x+3y \\
s.c. \quad & x+y \geq 1 \\
& x \geq 0, y\geq 0 \\
\end{align*}$$

First we construct the LP

In [ ]:
OPTIMIZER = GLPK.Optimizer              # We choose a solver
m = Model(with_optimizer(OPTIMIZER))    # We define a model

@variable(m,x>=0)                       # x is a positive variable of m
@variable(m,y>=0)                       # y is a positive variable of m

@objective(m,Min, 2*x+3*y)              # m's objective consists in Minimizing 2*x+3*y

@constraint(m,x+y >= 1 )                # m have the following constraint x+y <=1

We can check that we have written the right problem.

In [ ]:
print(m)

We can also solve m

In [ ]:
optimize!(m)
println(termination_status(m))
println(primal_status(m))
println(dual_status(m))

We can get the objective value and solution of m once solved.

In [ ]:
println(JuMP.objective_value(m))
println("x = ",JuMP.value(x))
println("y = ",JuMP.value(y))

### A more complex LP

We now consider
$$
\begin{align*}
\min_{x\in R^n} \quad & \sum_{i=1}^n c_i x_i \\
s.c. \quad & \sum_{i=1}^n x_i \geq n \\
&  -1 \leq x_i \leq 2 & \forall i
\end{align*}
$$

In [ ]:
m2 = Model(with_optimizer(OPTIMIZER))
n = 5                                    
c = rand(n)                               # c is random                                             

@variable(m2, -1<= x[1:n] <= 2)           # x is a vector of n variables of m2 : x[1], x[2],...,x[n] each between -1 and 2

@objective(m2,Min, sum(c[i]*x[i] for i=1:n) )

@constraint(m2,sum(x[i] for i=1:n) >= n)
                        
print(m2)            

In [ ]:
optimize!(m2)
println(termination_status(m2))

In [ ]:
value.(x)

We can now add multiple constraints
$$ x_i + x_{i+1} \leq 1, \qquad \forall i \in 2, \dots, n-1$$

In [ ]:
for i = 1 : n-1
    @constraint(m2, x[i]+x[i+1] <= 2)
end

In [ ]:
optimize!(m2)
value.(x)

### Declare multiple variables / constraints

We can declare variables and constraints by block with `@variables` and `@constraints` (note the `s`) as follows

In [ ]:
m3 = Model(with_optimizer(OPTIMIZER))

@variable(m3, -1<= x[1:n] <= 2)           
@objective(m3,Min, sum(c[i]*x[i] for i=1:n) )

@constraints(m3,begin 
        sum(x[i] for i=1:n) >= n         # first constraint
        [i in 1:n-1], x[i]+x[i+1] <= 2   # ∀ i = 1, .. n-1,  x_i + x_{i+1} <= 2
    end
    )

optimize!(m3)
value.(x)